<hr>

# 🤖 MACHINE LEARNING 🤖

<style>
h1 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

### 📂 IMPORTs


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [1]:
# install numpy and pandas if not already installed
!pip install numpy pandas

# install xgboost if not already installed
!pip install xgboost

# install shap if not already installed
!pip install shap

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

<hr>

## 📊 STEP 1 — Load Data


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

For consistency with code:
- X = features without the target variable; drop the target variable "revenue" from the features
- y = target variable "revenue"; the target is already binary (True=1 or False=0) 


In [3]:
import pandas as pd
# load the clean dataset
df = pd.read_csv("../data/processed/online_shoppers_intention_01_standard.csv")
print("Dataset loaded successfully. Shape:", df.shape)
display(df.head())

Dataset loaded successfully. Shape: (12330, 18)


,admin,admin_duration,info,info_duration,prod_related,prod_related_duration,bounce_rate,exit_rate,page_value,special_day,month,os,browser,region,traffic_type,visitor_type,weekend,revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


### Convert Boolean columns to int for consistency

In [4]:
# boolean columns
bool_cols = ["weekend", "revenue"]

# convert boolean columns to integers (0 and 1)
for col in bool_cols:
    df[col] = df[col].astype(int)

df.info() # check the data types

<class 'pandas.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   admin                  12330 non-null  int64  
 1   admin_duration         12330 non-null  float64
 2   info                   12330 non-null  int64  
 3   info_duration          12330 non-null  float64
 4   prod_related           12330 non-null  int64  
 5   prod_related_duration  12330 non-null  float64
 6   bounce_rate            12330 non-null  float64
 7   exit_rate              12330 non-null  float64
 8   page_value             12330 non-null  float64
 9   special_day            12330 non-null  float64
 10  month                  12330 non-null  str    
 11  os                     12330 non-null  int64  
 12  browser                12330 non-null  int64  
 13  region                 12330 non-null  int64  
 14  traffic_type           12330 non-null  int64  
 15  visitor_type 

### Separate Features (X) and Target (y)

In [9]:
# Features and target
X = df.drop("revenue", axis=1)  # features without the target variable "revenue"
y = df["revenue"]               # target variable "revenue"
print(f"Features and target variable separated. \nShapes: X =", X.shape, ", y =", y.shape)

Features and target variable separated. 
Shapes: X = (12330, 17) , y = (12330,)


<hr>

## ✂️ STEP 2 — Train/Test Split


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

Split the data into:
- Training set: X_train, y_train
- Test set :X_test, y_test
- Using stratify to maintain class balance


In [18]:
from sklearn.model_selection import train_test_split

# split the data into train/test sets 
# with stratification to maintain class balance

# Train sets:
# X_train    =     features for train
# y_train    =     target for train (one column with 0s and 1s)

# Test sets:
# X_test     =     features for test
# y_test     =     target for test (one column with 0s and 1s)


X_train, X_test, y_train, y_test = train_test_split(
    X,                  # features without the target variable "revenue"
    y,                  # target variable "revenue"
    test_size=0.2,      # 20% test set, 80% train set
    stratify=y,         # maintain class balance in splits
    random_state=42     # for reproducibility, we can choose any integer as the random state
)

print(f"Train/Test split completed. \nShapes: \nX_train = {X_train.shape}, y_train = {y_train.shape}, \nX_test = {X_test.shape}, y_test = {y_test.shape}")

display(X_train.head())
print(70*"-")
display(y_train.head())


Train/Test split completed. 
Shapes: 
X_train = (9864, 17), y_train = (9864,), 
X_test = (2466, 17), y_test = (2466,)


,admin,admin_duration,info,info_duration,prod_related,prod_related_duration,bounce_rate,exit_rate,page_value,special_day,month,os,browser,region,traffic_type,visitor_type,weekend
4263,8,732.25,0,0.0,34,1345.755952,0.005128,0.013342,12.274195,0.8,May,3,3,3,2,Returning_Visitor,0
5905,0,0.00,0,0.0,4,157.200000,0.040000,0.100000,0.000000,0.0,Oct,1,8,3,1,Returning_Visitor,1
9434,0,0.00,0,0.0,4,42.000000,0.000000,0.050000,0.000000,0.0,Dec,2,10,1,2,Returning_Visitor,0
3505,2,338.00,0,0.0,17,1205.566667,0.012500,0.037500,19.236250,0.8,May,3,2,2,2,Returning_Visitor,0
2067,0,0.00,0,0.0,32,827.646212,0.000000,0.000587,54.676348,0.0,Mar,2,2,7,2,Returning_Visitor,0


----------------------------------------------------------------------


4263    0
5905    0
9434    0
3505    1
2067    1
Name: revenue, dtype: int64

<hr>

## 🧹 STEP 3 — Preprocessing


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Define Categorical Columns ----------------------------------------------------------------

categorical_cols = [
    "month",
    "os",
    "browser",
    "region",
    "traffic_type",
    "visitor_type"
]

# Define Numerical columns (everything else)
numerical_cols = [col for col in X.columns if col not in categorical_cols]

# One-Hot Encoding --------------------------------------------------------------------------
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Fit ONLY on training data to avoid data leakage
encoder.fit(X_train[categorical_cols])

# Transform train and test
X_train_encoded = encoder.transform(X_train[categorical_cols])
X_test_encoded  = encoder.transform(X_test[categorical_cols])

# Convert to DataFrame
encoded_cols = encoder.get_feature_names_out(categorical_cols)

X_train_encoded_df = pd.DataFrame(
    X_train_encoded,
    columns=encoded_cols,
    index=X_train.index
)

X_test_encoded_df = pd.DataFrame(
    X_test_encoded,
    columns=encoded_cols,
    index=X_test.index
)

# Combine Numerical + Encoded Categorical -------------------------------------------------------------
X_train = pd.concat(
    [X_train[numerical_cols], X_train_encoded_df],
    axis=1
)

X_test = pd.concat(
    [X_test[numerical_cols], X_test_encoded_df],
    axis=1
)

# Save Train/Test sets in CSVs Files ----------------------------------------------------------------------
X_train.to_csv("../data/processed/X_train.csv", index=False)
y_train.to_csv("../data/processed/y_train.csv", index=False)
    
X_test.to_csv("../data/processed/X_test.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)

print("Preprocessing complete. Files saved successfully in ../data/processed/")

Preprocessing complete. Files saved successfully in ../data/processed/


<hr>

## 📊 STEP 4 — DISTIRBUTION ENTIRE/TRAIN/TEST


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
import pandas as pd

# assign:  X_train,  X_test,  y_train,  y_test with CSV files

X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv").squeeze()  # convert to Series
    
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv").squeeze()    # convert to Series

print(70*"-")

# calculate the class imbalance ratio for the training set 
# to use in XGBoost's scale_pos_weight parameter

# count number of 0s and 1s in training set
class_counts = y_train.value_counts()

ratio = class_counts[0] / class_counts[1]

print(f"Class imbalance ratio (negative:positive) in training set: {ratio:.2f}")
    # This ratio will be used in the XGBoost model to help it handle the class imbalance effectively.
    # A ratio greater than 1 indicates that the negative class (0) is more prevalent than the positive class (1),
    # which is common in many real-world datasets. 
    # By setting scale_pos_weight to this ratio, we can help XGBoost give more attention to the minority class during training.

print(70*"-")

# distribution ENTIRE dataset
print("Class distribution in entire dataset:")
print(pd.concat([y_train, y_test]).value_counts())

# percentage ENTIRE dataset
print(f"\nClass distribution percentages in entire dataset:")
print(pd.concat([y_train, y_test]).value_counts(normalize=True) * 100)

print(70*"-")

# distribution TRAIN set
print("Class distribution in training set:")
print(y_train.value_counts())

# percentage TRAIN set
print(f"\nClass distribution percentages in training set:")
print(y_train.value_counts(normalize=True) * 100)

print(70*"-")

# distribution TEST set
print("Class distribution in test set:")
print(y_test.value_counts())

# percentage TEST set
print(f"\nClass distribution percentages in test set:")
print(y_test.value_counts(normalize=True) * 100)

print(70*"-")

----------------------------------------------------------------------
Class imbalance ratio (negative:positive) in training set: 5.46
----------------------------------------------------------------------
Class distribution in entire dataset:
revenue
0    10422
1     1908
Name: count, dtype: int64

Class distribution percentages in entire dataset:
revenue
0    84.525547
1    15.474453
Name: proportion, dtype: float64
----------------------------------------------------------------------
Class distribution in training set:
revenue
0    8338
1    1526
Name: count, dtype: int64

Class distribution percentages in training set:
revenue
0    84.529603
1    15.470397
Name: proportion, dtype: float64
----------------------------------------------------------------------
Class distribution in test set:
revenue
0    2084
1     382
Name: count, dtype: int64

Class distribution percentages in test set:
revenue
0    84.509327
1    15.490673
Name: proportion, dtype: float64
------------------------

<hr>

## 📐 STEP 5 — ML MODEL RESULTS


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
import pandas as pd
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test = pd.read_csv("../data/processed/X_test.csv")   
y_train = pd.read_csv("../data/processed/y_train.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")

#### old model ?

In [ ]:
# define a dictionary of models with appropriate preprocessing and class imbalance handling ------------------------------------------
models = {

    # Logistic Regression can handle class imbalance by setting class_weight='balanced',
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Logistic Regression": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", LogisticRegression(class_weight='balanced', max_iter=1000))
    ]),
    
    # Linear SVM can handle class imbalance by setting class_weight='balanced', 
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Linear SVM": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", SVC(kernel='linear', class_weight='balanced', probability=True))
    ]),
    
    # KNN doesn't have a built-in way to handle class imbalance, but it can still be effective.
    # We will use the preprocessor with scaling for KNN as well, since KNN is sensitive to feature scales.
    "KNN": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", KNeighborsClassifier())
    ]),
    
    # Decision Tree can handle class imbalance by setting class_weight='balanced',
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Decision Tree": Pipeline([
        ("prep", preprocessor_tree),
        ("model", DecisionTreeClassifier(class_weight='balanced'))
    ]),
    
    # Random Forest can handle class imbalance by setting class_weight='balanced', 
    # which will automatically adjust weights inversely proportional to class frequencies.
    "Random Forest": Pipeline([
        ("prep", preprocessor_tree),
        ("model", RandomForestClassifier(class_weight='balanced'))
    ]),
    
    # XGBoost has a built-in way to handle class imbalance using the scale_pos_weight parameter.
    "XGBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", XGBClassifier(scale_pos_weight=ratio, eval_metric='logloss'))
    ]),

    # AdaBoost doesn't have a built-in way to handle class imbalance, but it can still be effective. 
    # We will use the preprocessor without scaling for AdaBoost as well.
    "AdaBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", AdaBoostClassifier())
    ])
}

print("Defined the following models with appropriate preprocessing and class imbalance handling:")
for model_name in models.keys():
    print(f" - {model_name}")

Defined the following models with appropriate preprocessing and class imbalance handling:
 - Logistic Regression
 - Linear SVM
 - KNN
 - Decision Tree
 - Random Forest
 - XGBoost
 - AdaBoost


#### run all models

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    matthews_corrcoef, confusion_matrix
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier


# 1️⃣ Define Feature Groups

numeric_features = [
    "admin", "admin_duration",
    "info", "info_duration",
    "prod_related", "prod_related_duration",
    "bounce_rate", "exit_rate",
    "page_value", "special_day"
]

# All remaining columns except target are binary (0/1)
binary_features = [col for col in X.columns if col not in numeric_features]


# 2️⃣ Define Preprocessors

# For linear models & KNN (need scaling)
preprocessor_scaled = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("bin", "passthrough", binary_features)
    ]
)

# For tree-based models (no scaling needed)
preprocessor_tree = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("bin", "passthrough", binary_features)
    ]
)


# 3️⃣ Class Imbalance Ratio

ratio = (y == 0).sum() / (y == 1).sum()


# 4️⃣ Define Models

models = {

    "Logistic Regression": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", LogisticRegression(
            class_weight='balanced',
            max_iter=5000,
            solver="lbfgs"
        ))
    ]),

    "Linear SVM": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", SVC(
            kernel='linear',
            class_weight='balanced',
            probability=True
        ))
    ]),

    "KNN": Pipeline([
        ("prep", preprocessor_scaled),
        ("model", KNeighborsClassifier())
    ]),

    "Decision Tree": Pipeline([
        ("prep", preprocessor_tree),
        ("model", DecisionTreeClassifier(
            class_weight='balanced',
            random_state=42
        ))
    ]),

    "Random Forest": Pipeline([
        ("prep", preprocessor_tree),
        ("model", RandomForestClassifier(
            class_weight='balanced',
            random_state=42
        ))
    ]),

    "XGBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", XGBClassifier(
            scale_pos_weight=ratio,
            eval_metric='logloss',
            use_label_encoder=False,
            random_state=42
        ))
    ]),

    "AdaBoost": Pipeline([
        ("prep", preprocessor_tree),
        ("model", AdaBoostClassifier(
            random_state=42
        ))
    ])
}


# 5️⃣ Cross Validation Setup

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []


# 6️⃣ Model Evaluation Loop

for name, model in models.items():

    precision_list = []
    recall_list = []
    f1_list = []
    roc_list = []
    pr_list = []
    mcc_list = []

    tp_total = tn_total = fp_total = fn_total = 0

    for train_idx, test_idx in cv.split(X, y):

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        precision_list.append(precision_score(y_test, y_pred))
        recall_list.append(recall_score(y_test, y_pred))
        f1_list.append(f1_score(y_test, y_pred))
        roc_list.append(roc_auc_score(y_test, y_proba))
        pr_list.append(average_precision_score(y_test, y_proba))
        mcc_list.append(matthews_corrcoef(y_test, y_pred))

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        tp_total += tp
        tn_total += tn
        fp_total += fp
        fn_total += fn


    results.append([
        name,
        np.mean(precision_list),
        np.mean(recall_list),
        np.mean(f1_list),
        np.sum(y == 1),
        np.mean(pr_list),
        np.mean(roc_list),
        np.mean(f1_list),
        np.std(f1_list),
        np.mean(mcc_list),
        tp_total, tn_total, fp_total, fn_total
    ])


# 7️⃣ Create Results Table

columns = [
    "Model", "Precision", "Recall", "F1 Score",
    "Support", "PR-AUC", "ROC-AUC",
    "Mean (F1)", "Std (F1)",
    "MCC", "TP", "TN", "FP", "FN"
]

results_df = pd.DataFrame(results, columns=columns)

print(results_df.sort_values(by="F1 Score", ascending=False))

C:\Users\sboub\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:200: UserWarning: [14:46:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\sboub\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:200: UserWarning: [14:46:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\sboub\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:200: UserWarning: [14:46:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\sboub\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:200: UserWarning: [14:46:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.

                 Model  Precision    Recall  F1 Score  Support    PR-AUC  \
5              XGBoost   0.610386  0.710695  0.656524     1908  0.715189   
1           Linear SVM   0.577485  0.754206  0.654103     1908  0.660935   
4        Random Forest   0.766972  0.512593  0.614272     1908  0.727298   
6             AdaBoost   0.679546  0.560789  0.614175     1908  0.663531   
0  Logistic Regression   0.505573  0.759963  0.607094     1908  0.650647   
3        Decision Tree   0.559100  0.541404  0.549836     1908  0.374395   
2                  KNN   0.647727  0.377371  0.476138     1908  0.483785   

    ROC-AUC  Mean (F1)  Std (F1)       MCC    TP    TN    FP    FN  
5  0.920603   0.656524  0.012740  0.589606  1356  9429   868   552  
1  0.909880   0.654103  0.013542  0.587370  1439  9244  1053   469  
4  0.924791   0.614272  0.014867  0.574193   978  9999   298   930  
6  0.915905   0.614175  0.019539  0.554357  1070  9792   505   838  
0  0.902459   0.607094  0.018866  0.532870  14

<hr>

## 🏆 STEP 6 —  Select TOP 3 Models


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

# 📊 PLOTS

<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<hr>

<hr>

## 01 - ROC Curves (Compare top 3 models)


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

In [ ]:
# paste the plot version you want to add to main notebook here

<hr>

## 02 - Adjust Decision Threshold


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- **Instead of 0.5:**
- **This should improve recall.**

In [ ]:
# paste the plot version you want to add to main notebook here

<hr>

## 03 - Feature Importance


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: center;
    color: hotpink;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- **For RF / XGBoost: Use  `.feature_importances_`**

In [25]:
import matplotlib.pyplot as plt

<hr>

# ML MODELS DEFINITONS & NOTES


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

---
#### NOTES
👉 Best model by ROC-AUC: XGBoost (0.9275)
- It has the highest ability to rank positive cases above negative cases.

⚠️ Is ROC-AUC enough for imbalanced data? Short answer: No, it is not enough.
- ROC-AUC can be misleading for imbalanced datasets because:
    - It evaluates performance across all classification thresholds.
    - It gives equal importance to True Positive Rate (TPR) and False Positive Rate (FPR).
    - When negatives heavily outnumber positives, the FPR can look artificially small, inflating ROC-AUC.
    - In highly imbalanced data, a model can achieve high ROC-AUC but still perform poorly at identifying the minority class.

🔎 For imbalanced datasets, prioritize:

1️⃣ Precision-Recall AUC (PR-AUC) ⭐ (Very Important)
- More informative when the positive class is rare.
- Focuses only on the positive class performance.

2️⃣ F1-score
- Harmonic mean of precision and recall.
- Good when you need balance between false positives and false negatives.

3️⃣ Recall (Sensitivity)
- Important if missing positives is costly (e.g., fraud, disease).

4️⃣ Confusion Matrix
- Shows actual classification behavior at your chosen threshold.

📊 Recommended Evaluation Metrics (Very Important)

Because the dataset is IMBALANCED:

✅ ROC-AUC
✅ Precision
✅ Recall
✅ F1-score
❌ Avoid relying only on accuracy

---
Order of modeling:
1. Logistic Regression (baseline)

2. Decision Tree

3. Random Forest

4. XGBoost

5. SVM

6. KNN

7. AdaBoost

---
🔬 Extra steps:
- Apply SMOTE before training
- Do feature importance comparison (RF vs XGBoost)
- Use cross-validation (StratifiedKFold)
- Plot ROC curves for all models

<hr>

# Basic / Linear Models


<style>
h1 {
    text-align: center;
    color: purple;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 1️⃣ Logistic Regression 


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- simple baseline for binary classification, interpretable.

**Why use it:**

- Strong baseline
- Interpretable (feature coefficients)
- Fast to train

**Tips for this dataset:**

- Use `class_weight='balanced'`
- Scale numeric features
- One-hot encode categorical features
- Evaluate with ROC-AUC and F1 (not accuracy)

<hr>

### 2️⃣ Linear SVM (Support Vector Machine)


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- good when classes are separable in feature space.

**Why use it:**

- Good for high-dimensional feature space
- Works well with proper scaling

**Tips:**

- Must scale features (StandardScaler)
- Use `class_weight='balanced'`
- Tune `C` parameter

<hr>

# Instance-Based


<style>
h1 {
    text-align: center;
    color: purple;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 3️⃣ K-Nearest Neighbors (KNN) 


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- simple distance-based classifier.

**Why use it:**

- Simple non-parametric model
- Captures non-linear boundaries

**Tips:**

- Feature scaling is mandatory
- Tune `k` (start with 5–25)
- Can be slow on larger datasets
- Sensitive to irrelevant features

<hr>

# Tree-Based & Ensemble Methods


<style>
h1 {
    text-align: center;
    color: purple;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

<hr>

### 4️⃣ Decision Trees


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- intuitive model, good for non-linear relationships.

**Why use it:**

- Handles non-linear relationships
- No need for feature scaling
- Easy to visualize

**Tune:**

- `max_depth`
- `min_samples_split`
- `min_samples_leaf`
- ⚠️ Can overfit — pruning is important.

<hr>

### 5️⃣ Random Forest


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- ensemble of trees; strong baseline with good performance on tabular data.

Why use it:

- Excellent for tabular data
- Handles feature interactions well
- Less overfitting than single tree

Tune:

- `n_estimators`
- `max_depth`
- `max_features`

🔥 A strong baseline for this dataset.

<hr>

### 6️⃣ Gradient Boosting Machines


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: black;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>
<hr>

- powerful boosted trees for classification.

<hr>

#### #️⃣ MODEL 06.1: XGBoost


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: grey;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: left;
    color: normal;
    font-weight: bold;
}
</style>
<hr>

- boosting method using weighted weak learners.

(Technically Gradient Boosting, but very powerful)

**Why use it:**
- State-of-the-art for structured/tabular data
- Handles imbalance well
- Often best performer

**Tune:**
- `learning_rate`
- `n_estimators`
- `max_depth`
- `scale_pos_weight` (important for imbalance)

🔥 Frequently achieves highest ROC-AUC.

<hr>

#### #️⃣ MODEL 06.2: AdaBoost


<style>
h1 {
    text-align: center;
    color: black;
    font-weight: bold;
}
</style>

<style>
h2 {
    text-align: left;
    color: red;
    font-weight: bold;
}
</style>

<style>
h3 {
    text-align: left;
    color: grey;
    font-weight: bold;
}
</style>

<style>
h4 {
    text-align: left;
    color: normal;
    font-weight: bold;
}
</style>
<hr>

- boosted trees that handle categorical features well.

**Why use it:**

- Boosting improves weak learners
- Simpler than XGBoost

**Tune:**
- `n_estimators`
- `learning_rate`

Usually performs better than a single tree but not as strong as Random Forest or XGBoost.